In [7]:
import os
from dotenv import load_dotenv
load_dotenv()

import operator
from typing import Annotated
from typing_extensions import TypedDict

# lets play with Tavily

In [9]:
from langchain_community.tools import TavilySearchResults

searchTool = TavilySearchResults(max_results=10)
user_query = "Who is the current Chief Minister of Delhi?"
search_results = searchTool.invoke(user_query)
search_results

[{'title': 'Delhi CM List: Chief Ministers of Delhi Party Names and ...',
  'url': 'https://www.jagranjosh.com/general-knowledge/delhi-chief-ministers-cm-list-party-name-and-tenure-1726561886-1',
  'content': "Rekha Gupta of the Bharatiya Janata Party (BJP) has been declared the new Chief Minister of Delhi. This announcement on February 19, 2025, marks the BJP's return to power in Delhi after 27 years, with the party securing 48 out of 70 assembly seats.\n\nGupta, previously the General Secretary of the Delhi BJP and a former president of the Delhi University Students' Union, won the Shalimar Bagh constituency. Her swearing-in ceremony is scheduled for February 20, 2025, at Ramlila Maidan. [...] Delhi CM List: Chief Ministers of Delhi Party Names and Tenure (1952-2025)\n\nDelhi Chief Ministers:\xa0Rekha Gupta is the 9th Chief Minister of Delhi, representing the Bharatiya Janata Party (BJP). The announcement was made on February 19, 2025, following the BJP's significant victory in the r

In [10]:
# generally these resutls are passed on to LLMs after concatinating the results

formatted_search_docs = "\n\n---\n\n".join(
    [
        f'<Document href="{doc["url"]}">\n{doc["content"]}\n</Document>'
        for doc in search_results
    ]
)

print(formatted_search_docs)

<Document href="https://www.jagranjosh.com/general-knowledge/delhi-chief-ministers-cm-list-party-name-and-tenure-1726561886-1">
Rekha Gupta of the Bharatiya Janata Party (BJP) has been declared the new Chief Minister of Delhi. This announcement on February 19, 2025, marks the BJP's return to power in Delhi after 27 years, with the party securing 48 out of 70 assembly seats.

Gupta, previously the General Secretary of the Delhi BJP and a former president of the Delhi University Students' Union, won the Shalimar Bagh constituency. Her swearing-in ceremony is scheduled for February 20, 2025, at Ramlila Maidan. [...] Delhi CM List: Chief Ministers of Delhi Party Names and Tenure (1952-2025)

Delhi Chief Ministers: Rekha Gupta is the 9th Chief Minister of Delhi, representing the Bharatiya Janata Party (BJP). The announcement was made on February 19, 2025, following the BJP's significant victory in the recent assembly elections, where they secured 48 out of 70 seats, marking their return to 

In [11]:
# then these formatted search results can be passed on to the LLM as context and it can be asked to retrieve information from this context. 

prompt_template = """
You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: {question} 
Context: {context} 
Answer:
"""

from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4o", temperature=0) 

prompt = prompt_template.format(context=formatted_search_docs, question=user_query)

response = llm.invoke(prompt)
response



AIMessage(content='The current Chief Minister of Delhi is Rekha Gupta, who took office on February 20, 2025.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 2654, 'total_tokens': 2677, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_76544d79cb', 'id': 'chatcmpl-BZx2MJsalDj227kPzdXnCOttDsXlR', 'finish_reason': 'stop', 'logprobs': None}, id='run-e6a70338-dd74-4fcc-b5c8-1f1c50f14846-0', usage_metadata={'input_tokens': 2654, 'output_tokens': 23, 'total_tokens': 2677, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [14]:
response.content

'The current Chief Minister of Delhi is Rekha Gupta, who took office on February 20, 2025.'